In [71]:
!pip install --upgrade git+https://github.com/flairNLP/flair.git

  Cloning https://github.com/flairNLP/flair.git to /tmp/pip-req-build-y4wy9jnw
  Running command git clone -q https://github.com/flairNLP/flair.git /tmp/pip-req-build-y4wy9jnw
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for flair: filename=flair-0.5-cp36-none-any.whl size=152787 sha256=5adcde0a87ca8aa74ed0c719ac3674b61439fe8c87139a0a895da9c25a515a88
  Stored in directory: /tmp/pip-ephem-wheel-cache-66hsv8we/wheels/84/82/73/d2b3b59b7be74ea05f2c6d64132efe27df52daffb47d1dc7bb
Successfully built flair
  Found existing installation: flair 0.5
    Uninstalling flair-0.5:
      Successfully uninstalled flair-0.5


In [0]:
from flair.data_fetcher import NLPTaskDataFetcher
from flair.embeddings import WordEmbeddings, FlairEmbeddings, DocumentLSTMEmbeddings, DocumentRNNEmbeddings, BertEmbeddings
from flair.models import TextClassifier
from flair.trainers import ModelTrainer
from pathlib import Path
from flair.datasets import CSVClassificationCorpus
from flair.data import Corpus
import pandas as pd

from flair.data import Sentence

In [73]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
data_folder = "./drive/My Drive/Colab Notebooks/capstone/data/"

### First Stage (Train on benchmark dataset)

In [0]:
benchmark = pd.read_csv(data_folder + "combined_benchmark.csv")

In [16]:
benchmark = benchmark[['label', 'text']]
benchmark.head()

,label,text
0,0,Why not subscribe to the magazine ?
1,-1,"Tornio Works employs 2,300 of whom more than 1..."
2,1,"The move is aimed at boosting sales , cost-eff..."
3,0,"As a result of the merger , the largest profes..."
4,-1,18 March 2010 A leakage in the gypsum pond was...


#### Create train, dev and test set

In [0]:
benchmark = benchmark.sample(frac=1, random_state=42)


benchmark.iloc[0:int(len(benchmark)*0.8)].to_csv(data_folder + 'train.csv', sep=',', index = False, header = False)
benchmark.iloc[int(len(benchmark)*0.8):int(len(benchmark)*0.9)].to_csv(data_folder + 'test.csv', sep=',', index = False, header = False)
benchmark.iloc[int(len(benchmark)*0.9):].to_csv(data_folder + 'dev.csv', sep=',', index = False, header = False)

In [0]:
# train_df = pd.read_csv(data_folder + "train.csv", header = None)
# train_df.head()

#### Build corpus

In [19]:
# corpus = NLPTaskDataFetcher.load_classification_corpus(Path(data_folder), test_file='test.csv', dev_file='dev.csv', train_file='train.csv')
column_name_map = {1: "text", 0: "label_topic"}

corpus: Corpus = CSVClassificationCorpus(data_folder,
                                         column_name_map,
                                         skip_header=False, #no header in kaggle data
                                         delimiter=',',    # comma separated rows
)

2020-06-11 20:46:21,815 Reading data from drive/My Drive/Colab Notebooks/capstone/data
2020-06-11 20:46:21,816 Train: drive/My Drive/Colab Notebooks/capstone/data/train.csv
2020-06-11 20:46:21,819 Dev: drive/My Drive/Colab Notebooks/capstone/data/dev.csv
2020-06-11 20:46:21,819 Test: drive/My Drive/Colab Notebooks/capstone/data/test.csv


#### Create word embeddings

In [20]:
word_embeddings = [BertEmbeddings(), FlairEmbeddings('news-forward-fast'), FlairEmbeddings('news-backward-fast')]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated method __init__. (Use 'TransformerWordEmbeddings' for all transformer-based word embeddings) -- Deprecated since version 0.4.5.
  """Entry point for launching an IPython kernel.


#### First Stage Fine-tuning

In [21]:
document_embeddings = DocumentLSTMEmbeddings(word_embeddings, hidden_size=512, reproject_words=True, reproject_words_dimension=256)
classifier = TextClassifier(document_embeddings, label_dictionary=corpus.make_label_dictionary(), multi_label=False)
trainer = ModelTrainer(classifier, corpus)
trainer.train(data_folder, max_epochs=10)

2020-06-11 20:46:32,348 Computing label dictionary. Progress:


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated method __init__. (The functionality of this class is moved to 'DocumentRNNEmbeddings') -- Deprecated since version 0.4.
  """Entry point for launching an IPython kernel.
100%|██████████| 1314/1314 [00:01<00:00, 807.68it/s]

2020-06-11 20:46:34,241 [b'0', b'1', b'-1']
2020-06-11 20:46:34,263 ----------------------------------------------------------------------------------------------------
2020-06-11 20:46:34,267 Model: "TextClassifier(
  (document_embeddings): DocumentLSTMEmbeddings(
    (embeddings): StackedEmbeddings(
      (list_embedding_0): BertEmbeddings(
        (model): BertModel(
          (embeddings): BertEmbeddings(
            (word_embeddings): Embedding(30522, 768, padding_idx=0)
            (position_embeddings): Embedding(512, 768)
            (token_type_embeddings): Embedding(2, 768)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (encoder): BertEncoder(
            (layer): ModuleList(
              (0): BertLayer(
                (attention): BertAttention(
                  (self): BertSelfAttention(
                    (query): Linear(in_features=768, out_features=768, bias=Tru

2020-06-11 20:46:35,936 epoch 1 - iter 3/37 - loss 1.72542675 - samples/sec: 71.55
2020-06-11 20:46:50,470 epoch 1 - iter 6/37 - loss 1.41298331 - samples/sec: 78.78
2020-06-11 20:47:02,726 epoch 1 - iter 9/37 - loss 1.38017842 - samples/sec: 77.69
2020-06-11 20:47:15,016 epoch 1 - iter 12/37 - loss 1.36771390 - samples/sec: 75.43
2020-06-11 20:47:28,746 epoch 1 - iter 15/37 - loss 1.29182448 - samples/sec: 70.53
2020-06-11 20:47:41,178 epoch 1 - iter 18/37 - loss 1.26792369 - samples/sec: 74.70
2020-06-11 20:47:53,446 epoch 1 - iter 21/37 - loss 1.27641764 - samples/sec: 76.76
2020-06-11 20:48:05,801 epoch 1 - iter 24/37 - loss 1.25897953 - samples/sec: 79.21
2020-06-11 20:48:17,829 epoch 1 - iter 27/37 - loss 1.22091764 - samples/sec: 81.07
2020-06-11 20:48:30,202 epoch 1 - iter 30/37 - loss 1.20964215 - samples/sec: 79.54
2020-06-11 20:48:42,165 epoch 1 - iter 33/37 - loss 1.19802785 - samples/sec: 84.35
2020-06-11 20:48:54,258 epoch 1 - iter 36/37 - loss 1.20336205 - samples/sec: 8

{'dev_loss_history': [0.8829440474510193,
  0.7203056216239929,
  0.9014942049980164,
  0.7286069393157959,
  0.8807229995727539,
  0.6408273577690125,
  0.7970978617668152,
  0.6934536695480347,
  0.8263334631919861,
  0.8744912147521973],
 'dev_score_history': [0.7989,
  0.8622,
  0.8077,
  0.8522,
  0.8099,
  0.8673,
  0.8497,
  0.8647,
  0.8448,
  0.8497],
 'test_score': 0.8439,
 'train_loss_history': [1.2002636323104034,
  0.9350489458522281,
  0.8268443056055017,
  0.8040031033593256,
  0.7500786281920768,
  0.6970284959754428,
  0.7145128443434432,
  0.6085004999830916,
  0.664192912546364,
  0.5949043977904964]}

### Second Stage (train on hand annotated datasets)

#### Build corpus

In [24]:
new_data_folder = "./drive/My Drive/Colab Notebooks/capstone/data/phase_2_mortgage_rate_oversampled/"
new_column_name_map = {5: "text", 4: "label_topic"}

corpus: Corpus = CSVClassificationCorpus(new_data_folder,
                                         new_column_name_map,
                                         skip_header=True, #yes header in 
                                         delimiter=',',    # comma separated rows
)

2020-06-11 21:14:19,317 Reading data from drive/My Drive/Colab Notebooks/capstone/data/phase_2_mortgage_rate_oversampled
2020-06-11 21:14:19,318 Train: drive/My Drive/Colab Notebooks/capstone/data/phase_2_mortgage_rate_oversampled/train.csv
2020-06-11 21:14:19,319 Dev: drive/My Drive/Colab Notebooks/capstone/data/phase_2_mortgage_rate_oversampled/dev.csv
2020-06-11 21:14:19,319 Test: drive/My Drive/Colab Notebooks/capstone/data/phase_2_mortgage_rate_oversampled/test.csv


#### Second Stage fine-tuning

In [25]:


benchmark_classifier = TextClassifier.load(data_folder + 'best-model.pt')

2020-06-11 21:14:23,960 loading file ./drive/My Drive/Colab Notebooks/capstone/data/best-model.pt


In [26]:
trainer = ModelTrainer(benchmark_classifier, corpus)
trainer.train(new_data_folder, max_epochs=10)


2020-06-11 21:14:39,141 ----------------------------------------------------------------------------------------------------
2020-06-11 21:14:39,146 Model: "TextClassifier(
  (document_embeddings): DocumentLSTMEmbeddings(
    (embeddings): StackedEmbeddings(
      (list_embedding_0): BertEmbeddings(
        (model): BertModel(
          (embeddings): BertEmbeddings(
            (word_embeddings): Embedding(30522, 768, padding_idx=0)
            (position_embeddings): Embedding(512, 768)
            (token_type_embeddings): Embedding(2, 768)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (encoder): BertEncoder(
            (layer): ModuleList(
              (0): BertLayer(
                (attention): BertAttention(
                  (self): BertSelfAttention(
                    (query): Linear(in_features=768, out_features=768, bias=True)
                    (key): Linear(in_feat

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


saving best model
2020-06-11 21:15:53,968 ----------------------------------------------------------------------------------------------------
2020-06-11 21:15:54,846 epoch 2 - iter 1/6 - loss 1.21896505 - samples/sec: 63.88
2020-06-11 21:16:08,170 epoch 2 - iter 2/6 - loss 1.34578860 - samples/sec: 80.09
2020-06-11 21:16:19,948 epoch 2 - iter 3/6 - loss 1.22393487 - samples/sec: 73.39
2020-06-11 21:16:31,931 epoch 2 - iter 4/6 - loss 1.12664030 - samples/sec: 86.30
2020-06-11 21:16:43,396 epoch 2 - iter 5/6 - loss 1.10699414 - samples/sec: 78.26
2020-06-11 21:16:54,503 epoch 2 - iter 6/6 - loss 1.14799018 - samples/sec: 331.43
2020-06-11 21:17:05,857 ----------------------------------------------------------------------------------------------------
2020-06-11 21:17:05,858 EPOCH 2 done: loss 1.1480 - lr 0.1000000
2020-06-11 21:17:06,526 DEV : loss 1.2450100183486938 - score 0.7353
2020-06-11 21:17:06,559 BAD EPOCHS (no improvement): 0
saving best model
2020-06-11 21:17:08,425 --------

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'dev_loss_history': [1.3760476112365723,
  1.2450100183486938,
  1.7610528469085693,
  0.8523023128509521,
  0.8766542077064514,
  0.8038918375968933,
  1.3781912326812744,
  2.0921192169189453,
  1.024451732635498,
  1.8005850315093994],
 'dev_score_history': [0.6944,
  0.7353,
  0.7692,
  0.8333,
  0.8621,
  0.8772,
  0.7692,
  0.7353,
  0.8333,
  0.8197],
 'test_score': 0.8772,
 'train_loss_history': [1.5251025954882305,
  1.1479901770750682,
  0.7269907395044962,
  0.9134433368841807,
  0.5661698778470358,
  0.675675223271052,
  0.46617567042509717,
  0.4531128679712613,
  0.41995643575986225,
  0.16242523243029913]}

In [82]:
mortgage_classifier = TextClassifier.load(new_data_folder + '/best-model.pt')

2020-06-11 22:56:19,218 loading file ./drive/My Drive/Colab Notebooks/capstone/data/phase_2_mortgage_rate_oversampled//best-model.pt


In [92]:
sentence = Sentence("Canada's growth weakens as investment drops, consumers fade")

mortgage_classifier.predict(sentence, multi_class_prob=True)

print(sentence.labels) ## incorrect, but uncertain

[-1 (0.4634)]


In [0]:
pred_df = pd.read_csv(data_folder + "mortgage_rates_CBC_article_to_predict.csv")

In [0]:
pred_df.head()

In [0]:
def finetuned_model_predictions(input_file_path, finetuned_classifier, output_file_path):
  '''Makes Sentiment Predictions on unannotated data points contained in the input csvfile by loading the user-defined classifier.
     Exports the csvfile by adding two columns 'pred_label' and 'confidence' and filling in results from model predictions.
  '''
  unannotated_df = pd.read_csv(input_file_path)

  unannotated_df['best_label'] = None
  unannotated_df['best_confidence'] = None
  unannotated_df['second_likely'] = None
  unannotated_df['second_confidence'] = None
  unannotated_df['least_likely'] = None
  unannotated_df['least_confidence'] = None
  for i in range(len(unannotated_df)):
    #print(unannotated_df['title_desc'].iloc[i])
    sentence = Sentence(unannotated_df['title_desc'].iloc[i])
    finetuned_classifier.predict(sentence,  multi_class_prob=True)
    print(sentence.labels)
    pred_score_label = [(sentence.labels[c].score, sentence.labels[c].value) for c in range(len(sentence.labels))]
    pred_score_label.sort()
    print(pred_score_label)
    # list in ascending order on confidence score
    best_label = int(pred_score_label[-1][1])
    best_confidence = pred_score_label[-1][0]
    second_likely_label = int(pred_score_label[-2][1]) 
    second_likely_confidence = pred_score_label[-2][0]
    least_likely_label = int(pred_score_label[0][1]) 
    least_likely_confidence = pred_score_label[0][0]

    unannotated_df['best_label'].iloc[i] = best_label
    unannotated_df['best_confidence'].iloc[i] = best_confidence
    unannotated_df['second_likely'].iloc[i] = second_likely_label
    unannotated_df['second_confidence'].iloc[i] = second_likely_confidence
    unannotated_df['least_likely'].iloc[i] = least_likely_label
    unannotated_df['least_confidence'].iloc[i] = least_likely_confidence

  print(f"All { len(unannotated_df) } rows done prediction! ")
  unannotated_df.to_csv(output_file_path,index=False)
  print("Done export!")

  return unannotated_df['best_label'].value_counts()



In [78]:
# #BLOOMBERG - MAKE PREDICTIONS W ALL PROBS
# input_name = data_folder + "predictions_dataset_mortgagerates_Bloomberg.csv"
# print(input_name)
# output_name = data_folder + "unannotated_mortgagerates_Bloomberg_predictions.csv"
# print(output_name)

#finetuned_model_predictions(input_name, mortgage_classifier, output_name)

./drive/My Drive/Colab Notebooks/capstone/data/predictions_dataset_mortgagerates_Bloomberg.csv
./drive/My Drive/Colab Notebooks/capstone/data/unannotated_mortgagerates_Bloomberg_predictions.csv


In [0]:
#CBC - MAKE PREDICTIONS W ALL PROBS
input_name = data_folder + "predictions_dataset_mortgagerates_cbc.csv"
print(input_name)
output_name = data_folder + "unannotated_mortgagerates_CBC_predictions.csv"
print(output_name)

finetuned_model_predictions(input_name, mortgage_classifier, output_name)

./drive/My Drive/Colab Notebooks/capstone/data/predictions_dataset_mortgagerates_cbc.csv
./drive/My Drive/Colab Notebooks/capstone/data/unannotated_mortgagerate_CBC_predictions.csv


In [0]:
pred_list = []
conf_list = []

for index, row in pred_df.iterrows():
  article_text = row["title_desc"]
  article_sentence = Sentence(article_text)
  mortgage_classifier.predict(article_sentence)
  pred = article_sentence.labels[0].value
  print(pred)
  pred_list.append(pred)

  conf_score = article_sentence.labels[0].score
  print(conf_score)
  conf_list.append(conf_score)

  print('----')


0
0.5833262801170349
----
0
0.8457203507423401
----
-1
0.3939833641052246
----
-1
0.7230633497238159
----
-1
0.7956459522247314
----
-1
0.5243971347808838
----
0
0.8241885900497437
----
-1
0.5161066651344299
----
0
0.9054300785064697
----
0
0.5838729739189148
----
-1
0.9202001690864563
----
-1
0.7700586915016174
----
-1
0.7929283380508423
----
-1
0.5186101198196411
----
-1
0.7283235192298889
----
0
0.7462179660797119
----
0
0.9179391264915466
----
-1
0.723307728767395
----
-1
0.723307728767395
----
0
0.9848251938819885
----
-1
0.6162759065628052
----
0
0.8945677876472473
----
-1
0.6524528861045837
----
-1
0.8799123167991638
----
-1
0.61030113697052
----
-1
0.61030113697052
----
0
0.9861264824867249
----
0
0.987683117389679
----
-1
0.6217166185379028
----
0
0.4838629364967346
----
0
0.8227150440216064
----
-1
0.5220191478729248
----
0
0.8764476776123047
----
0
0.9738495945930481
----
0
0.7048612236976624
----
0
0.9464749097824097
----
-1
0.6920495629310608
----
0
0.9608901143074036
----

In [0]:
print(len(conf_list))
print(len(pred_list))

330
330


In [0]:

pred_df["pred_label"] = pd.Series(pred_list)
pred_df["confidence"] = pd.Series(conf_list)  

In [0]:
pred_df.head()

,Unnamed: 0,source,title_desc,publishedAt,pred_label,confidence
0,0,CBC,Mortgage arrears rate could spike to double wh...,2020-05-14,0,0.583326
1,1,CBC,Interest rates are plunging — so why aren't mo...,2020-04-04,0,0.845720
2,2,CBC,Why worries about the coronavirus are pushing ...,2020-01-29,-1,0.393983
3,3,CBC,U.S. Fed chair rules out negative interest rat...,2020-05-13,-1,0.723063
4,4,CBC,Mortgages in arrears in Alberta hit highest ra...,2019-08-14,-1,0.795646


In [0]:
for index, row in pred_df.iterrows():
  print(row["title_desc"])
  print(row["pred_label"])

Mortgage arrears rate could spike to double what it was in 2009, Bank of Canada says. Central bank says number of people falling behind on mortgages could almost quadruple
0
Interest rates are plunging — so why aren't mortgage rates?. Bond yields and central bank rates have never been lower, but not all the savings are filtering down
0
Why worries about the coronavirus are pushing mortgage rates down. Fear has pushed investors to buy up bonds, which is causing cheaper borrowing for home buyers
-1
U.S. Fed chair rules out negative interest rates even as Trump trumpets them. U.S. president goes negative on Jerome Powell for rejection of below-zero interest rates
-1
Mortgages in arrears in Alberta hit highest rate since 2013. Rate seen as a 'lagging indicator' of the economy, driven largely by unemployment
-1
'Pretty cheap money': Canadian mortgage rates falling to their lowest level in 2 years. Fixed and variable loans have gotten cheaper because costs for lenders are down too
-1
Housing

In [0]:
out_filepath = data_folder + "unannotated_mortgagerate_CBC_predictions.csv"
print(out_filepath)

./drive/My Drive/Colab Notebooks/capstone/data//unannotated_mortgagerate_CBC_predictions.csv


In [0]:
pred_df.to_csv(out_filepath,index=False)

In [0]:
mort_test_df = pd.read_csv(data_folder + "/test.csv", header=None)

mort_test_df.head()
  

,0,1
0,-1,Finnish metal products company Componenta Oyj ...
1,1,Incap and Lankapaja aim to enter into actual a...
2,-1,Turkish stocks tumble as crackdown on lira spe...
3,1,"Summer Winds Down, and Big Tech Is Called Befo..."
4,-1,Here's why the slump in semiconductor stocks m...


In [0]:
correct = 0
severe_incorrect = 0
for i in range(len(corpus.test)):
  print(i)
  sentence = Sentence(mort_test_df.iloc[i,1])
  mortgage_classifier.predict(sentence)
  if sentence.labels[0].value == corpus.test[i].labels[0].value:
    correct += 1
  else:

    print("INCORRECT CLASSIFICATION: ")
    print("TEXT: ", sentence)
    print("TRUE LABEL: ", corpus.test[i].labels[0].value)
    print("PRED: ",sentence.labels[0].value)
    
    if corpus.test[i].labels[0].value != "0" and sentence.labels[0].value != "0":
      severe_incorrect += 1


  print("----")


0
----
1
----
2
----
3
INCORRECT CLASSIFICATION: 
TEXT:  Sentence: "Summer Winds Down, and Big Tech Is Called Before Congress. The newest jobs numbers are expected to show healthy gains, and Detroit automakers will report sales for August."   [− Tokens: 28  − Sentence-Labels: {'class': [0 (0.5657)]}]
TRUE LABEL:  -1
PRED:  0
----
4
----
5
----
6
INCORRECT CLASSIFICATION: 
TEXT:  Sentence: "Many listed stocks currently undervalued. More than half of listed stocks here are trading at prices below "book value" . suited for valuing capital-intensive companies or financial businesses with a lot of ."   [− Tokens: 33  − Sentence-Labels: {'class': [1 (0.4755)]}]
TRUE LABEL:  0
PRED:  1
----
7
INCORRECT CLASSIFICATION: 
TEXT:  Sentence: "Revenue grew 1 percent to euro742 .2 million US$ 964 million from euro735 million ."   [− Tokens: 15  − Sentence-Labels: {'class': [0 (0.8079)]}]
TRUE LABEL:  1
PRED:  0
----
8
INCORRECT CLASSIFICATION: 
TEXT:  Sentence: "The Point Village , designed by Scott 

In [0]:

print("CORRECT: ",correct/len(corpus.test))
print("INCORRECT: ",1 - correct/len(corpus.test))
print("OPPOSITE PREDICTIONS (SEVERELY INCORRECT): ",severe_incorrect/len(corpus.test))

CORRECT:  0.68
INCORRECT:  0.31999999999999995
OPPOSITE PREDICTIONS (SEVERELY INCORRECT):  0.0


In [0]:
#Approximately 70% correct
#In incorrect labels, most had a true label of negative but were overfit to be neutral

#Not many positive news examples likely means that accuracy in positive tags is not that high